In [1]:
from pymongo import MongoClient
import csv
import time
import pandas as pd

In [2]:
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ProjectIF29
collection = db['Tweets']

### Generer les attributs du graphe social 

**_id**: id du user\
**friends_count** : nombre de profils suivis\
**followers_count**: nombre de followers ou d'abonnees\
**listed_count**: le nombre de mention publique pour cet utilisateur\
**statuses_count**: le nombre de tweet et retweet emis par le user\
**favourites_count**: nombre de favoris ou nombre de tweets likés depuis la creation du compte \
**accounted_created_time**: jour et heure de la creation du compte\
**tweet_created_time** : jour et heure de la creation du tweet

In [28]:

collection = db['tweet']
dictdata = collection.aggregate(
    [
        {"$group" : {
            "_id" : "$user.id", 
            "friends_count" : {"$last" : "$user.friends_count"},
            "listed_count" : {"$last" : "$user.listed_count"},
            "favourites_count" : {"$last" : "$user.favourites_count"},
            "statuses_count" : {"$last" : "$user.statuses_count"},
            "followers_count" : {"$last" : "$user.followers_count"},
            "account_created_time" : {"$last" : "$user.created_at"},
            "tweet_created_time" : {"$first": "$created_at"}
        }
        }
    ]

entetes = ['_id', 'friends_count', 'listed_count', 'favourites_count', 
           'statuses_count', 'followers_count', 'account_created_time', 'tweet_created_time']
#Enregistrer le curseur dans un fichier  csv
with open('Les_attributs.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=entetes)
    writer.writeheader()
    writer.writerows(dictdata)

### Recuperation des textes des tweets

In [59]:
collection = db['tweet']
cursor1 = collection.aggregate(
    [
        {"$group" : {
            "_id" : "$user.id", 
            "Les_tweets" : {"$push": "$text"}
        }
        }
    ]
)
entetes = ['_id', 'Les_tweets']
with open('Les_tweets.csv', 'w', newline ='', encoding = 'utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=entetes)
    writer.writeheader()
    writer.writerows(cursor1)

In [11]:
def frequencePublication(data):
    #la conversion en heure du tweet
    t0 = time.mktime( time.strptime(data["tweet_created_time"], "%a %b %d %H:%M:%S +0000 %Y"))
    t1 = time.mktime(time.strptime(data["account_created_time"], '%a %b %d %H:%M:%S +0000 %Y'))
    return data["statuses_count"]/(t0 - t1)*100
data = pd.read_csv('nombre.csv')
data['frequence_statues'] = data.apply(frequencePublication, axis = 1)
data

,_id,friends_count,listed_count,favourites_count,statuses_count,followers_count,account_created_time,tweet_created_time,frequence_statues
0,3.398963e+08,528,0,560,7079,1315,Thu Jul 21 20:47:54 +0000 2011,Thu Jun 14 02:14:29 +0000 2018,0.003252
1,7.233354e+17,202,12,7555,7602,302,Fri Apr 22 02:19:25 +0000 2016,Thu Jun 14 02:15:30 +0000 2018,0.011237
2,1.321441e+09,223,24,64422,248761,1535,Tue Apr 02 00:49:32 +0000 2013,Thu Jun 14 02:16:02 +0000 2018,0.151611
3,2.250695e+09,3293,2,158,2477,1457,Sun Dec 29 19:44:52 +0000 2013,Thu Jun 14 02:18:22 +0000 2018,0.001762
4,1.571479e+09,302,1,2371,12415,434,Fri Jul 05 22:50:12 +0000 2013,Thu Jun 14 02:16:34 +0000 2018,0.007965
...,...,...,...,...,...,...,...,...,...
1698,3.998702e+08,687,0,232,250,151,Fri Oct 28 05:03:49 +0000 2011,Thu Jun 14 02:20:46 +0000 2018,0.000120
1699,2.824046e+08,768,0,7072,3217,161,Fri Apr 15 04:22:06 +0000 2011,Thu Jun 14 02:17:17 +0000 2018,0.001423
1700,8.695284e+17,1034,0,6300,712,98,Tue May 30 12:18:25 +0000 2017,Thu Jun 14 02:20:47 +0000 2018,0.002171
1701,1.499489e+09,214,36,47,142274,2308,Mon Jun 10 21:41:46 +0000 2013,Thu Jun 14 02:22:34 +0000 2018,0.090023


In [20]:
def frequenceFriends(data):
    #la conversion en heure du tweet
    t0 = time.mktime( time.strptime(data["tweet_created_time"], "%a %b %d %H:%M:%S +0000 %Y"))
    t1 = time.mktime(time.strptime(data["account_created_time"], '%a %b %d %H:%M:%S +0000 %Y'))
    return data["friends_count"]/(t0 - t1)*100
data['frequence_friends'] = data.apply(frequenceFriends, axis = 1)
data

,_id,friends_count,listed_count,favourites_count,statuses_count,followers_count,account_created_time,tweet_created_time,frequence_statues,ratio,frequence_friends
0,3.398963e+08,528,0,560,7079,1315,Thu Jul 21 20:47:54 +0000 2011,Thu Jun 14 02:14:29 +0000 2018,0.003252,2.490530,0.000243
1,7.233354e+17,202,12,7555,7602,302,Fri Apr 22 02:19:25 +0000 2016,Thu Jun 14 02:15:30 +0000 2018,0.011237,1.495050,0.000299
2,1.321441e+09,223,24,64422,248761,1535,Tue Apr 02 00:49:32 +0000 2013,Thu Jun 14 02:16:02 +0000 2018,0.151611,6.883408,0.000136
3,2.250695e+09,3293,2,158,2477,1457,Sun Dec 29 19:44:52 +0000 2013,Thu Jun 14 02:18:22 +0000 2018,0.001762,0.442454,0.002342
4,1.571479e+09,302,1,2371,12415,434,Fri Jul 05 22:50:12 +0000 2013,Thu Jun 14 02:16:34 +0000 2018,0.007965,1.437086,0.000194
...,...,...,...,...,...,...,...,...,...,...,...
1698,3.998702e+08,687,0,232,250,151,Fri Oct 28 05:03:49 +0000 2011,Thu Jun 14 02:20:46 +0000 2018,0.000120,0.219796,0.000328
1699,2.824046e+08,768,0,7072,3217,161,Fri Apr 15 04:22:06 +0000 2011,Thu Jun 14 02:17:17 +0000 2018,0.001423,0.209635,0.000340
1700,8.695284e+17,1034,0,6300,712,98,Tue May 30 12:18:25 +0000 2017,Thu Jun 14 02:20:47 +0000 2018,0.002171,0.094778,0.003153
1701,1.499489e+09,214,36,47,142274,2308,Mon Jun 10 21:41:46 +0000 2013,Thu Jun 14 02:22:34 +0000 2018,0.090023,10.785047,0.000135


### ratio nombre de followers et nombre profils suivi

In [12]:
def division(df):
    if df['friends_count'] !=0:
        return df['followers_count']/df['friends_count']
    else:
        return 0
data['ratio'] = data.apply(division, axis=1)

In [13]:
data['ratio']==0

0       False
1       False
2       False
3       False
4       False
        ...  
1698    False
1699    False
1700    False
1701    False
1702    False
Name: ratio, Length: 1703, dtype: bool

### Recherche de # et @

In [14]:
def hashtag(df):
    diese = df['Les_tweets'].count('#')
    arobaz = df['Les_tweets'].count('@')
    visibile= (diese*11.6 + arobaz*11.4)/140
    return visibile
df = pd.read_csv('Les_tweets.csv')
df['visibilite'] = df.apply(hashtag, axis=1)

In [15]:
df

,_id,Les_tweets,visibilite
0,7.712831e+08,['Finally. #WorldCup'],0.082857
1,3.361040e+08,"[""RT @CadreFloyd: Some of the 'men' who'll be ...",0.164286
2,4.165152e+07,['RT @CBSThisMorning: 2026 #WorldCup announcem...,0.245714
3,9.067093e+17,['RT @billboard: Shawn Mendes drops new versio...,0.164286
4,5.628488e+07,['Who you got in the #WorldCup@@Briana_Lee_xx ...,0.245714
...,...,...,...
1698,1.000244e+18,['RT @kialatino: Cuándo sabes que la #CopaMund...,0.330000
1699,1.265995e+09,['So funny. #WorldCup https://t.co/g8bTp0CZwZ'],0.082857
1700,1.941272e+09,['RT @BBST15: Skuad Germany🇩🇪 untuk Piala Duni...,0.247143
1701,2.938594e+09,['RT @SaudiNT: هز الشباك.. جمهورك وراك! 😍💚\n\n...,0.330000


### Rassemblement des labels

In [21]:
labels = pd.DataFrame()
labels['frequence_statuses'] = data['frequence_statues']
labels['visibilite'] = df['visibilite']
labels['frequence_friends'] = data['frequence_friends']

In [22]:
labels

,frequence_statuses,visibilite,frequence_friends
0,0.003252,0.082857,0.000243
1,0.011237,0.164286,0.000299
2,0.151611,0.245714,0.000136
3,0.001762,0.164286,0.002342
4,0.007965,0.245714,0.000194
...,...,...,...
1698,0.000120,0.330000,0.000328
1699,0.001423,0.082857,0.000340
1700,0.002171,0.247143,0.003153
1701,0.090023,0.330000,0.000135
